In [2]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import os

# Load tickers from all sources (assuming you've combined them already)
reddit = pd.read_csv("/home/rupam/DataAlpha/data/raw/processed/reddit_sentiment_2020.csv")
google = pd.read_csv("/home/rupam/DataAlpha/data/raw/processed/google_trends_sentiment_2020.csv")
news = pd.read_csv("/home/rupam/DataAlpha/data/raw/processed/news_sentiment_2020.csv")

# Combine tickers
all_tickers = set(reddit["ticker"]).union(set(google["ticker"])).union(set(news["ticker"]))
print(f"✅ Total unique tickers to fetch: {len(all_tickers)}")

# Set output path
os.makedirs("/home/rupam/DataAlpha/data/raw/Stock_prices", exist_ok=True)

# Fetch and save each ticker's OHLC data
failed = []
for ticker in tqdm(all_tickers, desc="📉 Fetching OHLC data"):
    try:
        df = yf.download(ticker, start="2020-01-01", end="2020-12-31")
        if df.empty:
            failed.append(ticker)
            continue
        df.reset_index(inplace=True)
        df["ticker"] = ticker
        df.to_csv(f"/home/rupam/DataAlpha/data/raw/Stock_prices/{ticker}.csv", index=False)
    except Exception as e:
        print(f"❌ Error fetching {ticker}: {e}")
        failed.append(ticker)

print(f"✅ Completed. Failed tickers: {len(failed)}")


✅ Total unique tickers to fetch: 493


📉 Fetching OHLC data:   0%|          | 0/493 [00:00<?, ?it/s]/tmp/ipykernel_226624/1795103521.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2020-12-31")
[*********************100%***********************]  1 of 1 completed
📉 Fetching OHLC data:   0%|          | 1/493 [00:01<14:54,  1.82s/it]/tmp/ipykernel_226624/1795103521.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2020-12-31")
[*********************100%***********************]  1 of 1 completed
📉 Fetching OHLC data:   0%|          | 2/493 [00:03<12:21,  1.51s/it]/tmp/ipykernel_226624/1795103521.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2020-01-01", end="2020-12-31")
[*********************100%***********************]  1 of 1 completed
📉 Fetching OHLC data:   1%|          | 3/493 

✅ Completed. Failed tickers: 5


In [3]:
failed

['SW', 'COIN', 'GEHC', 'KVUE', 'EXE']

In [11]:
import os
import pandas as pd
from tqdm import tqdm

# Input/output folders
price_folder = "/home/rupam/DataAlpha/data/raw/Stock_prices"
output_folder = "/home/rupam/DataAlpha/data/raw/returns"
os.makedirs(output_folder, exist_ok=True)

# List all CSV files
csv_files = [f for f in os.listdir(price_folder) if f.endswith(".csv")]
print(f"📁 Found {len(csv_files)} stock files")

failed = []

for file in tqdm(csv_files, desc="📊 Calculating returns"):
    try:
        file_path = os.path.join(price_folder, file)

        # ✅ Read with correct headers (row 0) and skip row 1 (ticker names)
        df = pd.read_csv(file_path, skiprows=[1])

        # Ensure required columns exist
        if "Date" not in df.columns or "Close" not in df.columns:
            raise ValueError("Missing required columns")

        # Convert and sort dates
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df = df.dropna(subset=["Date"])
        df = df.sort_values("Date")

        # Calculate daily return
        df["return"] = df["Close"].pct_change()

        # Add ticker from filename
        df["ticker"] = file.replace(".csv", "")

        # Drop NaN returns
        df = df.dropna(subset=["return"])

        # Keep only required columns
        df = df[["Date", "ticker", "return"]]
        df.rename(columns={"Date": "date"}, inplace=True)

        # Save to output
        df.to_csv(os.path.join(output_folder, file), index=False)

    except Exception as e:
        print(f"❌ Failed: {file} -> {e}")
        failed.append(file)

print(f"✅ Returns calculated for {len(csv_files) - len(failed)} tickers")


📁 Found 488 stock files


📊 Calculating returns: 100%|██████████| 488/488 [00:03<00:00, 132.60it/s]

✅ Returns calculated for 488 tickers


In [13]:
import os
import pandas as pd
from tqdm import tqdm

# Paths
returns_folder = "/home/rupam/DataAlpha/data/raw/returns"
output_path = "/home/rupam/DataAlpha/data/raw/processed/all_returns.csv"

# List all return CSVs
return_files = [f for f in os.listdir(returns_folder) if f.endswith(".csv")]

# Collect DataFrames
all_returns = []

for file in tqdm(return_files, desc="📦 Merging return files"):
    try:
        df = pd.read_csv(os.path.join(returns_folder, file), parse_dates=["date"])
        all_returns.append(df)
    except Exception as e:
        print(f"❌ Failed: {file} -> {e}")

# Combine all returns
df_returns = pd.concat(all_returns, ignore_index=True)

# Save combined returns
df_returns.to_csv(output_path, index=False)
print(f"✅ Merged returns saved to: {output_path}")


📦 Merging return files: 100%|██████████| 488/488 [00:01<00:00, 486.86it/s]


✅ Merged returns saved to: /home/rupam/DataAlpha/data/raw/processed/all_returns.csv


In [17]:
import pandas as pd
import os

# Load all pre-processed CSVs
returns_path = "/home/rupam/DataAlpha/data/raw/returns"
reddit_path = "/home/rupam/DataAlpha/data/raw/processed/reddit_sentiment_2020.csv"
news_path = "/home/rupam/DataAlpha/data/raw/processed/news_sentiment_2020.csv"
google_path = "/home/rupam/DataAlpha/data/raw/processed/google_trends_sentiment_2020.csv"

# Load returns (all files and concatenate)
all_returns = []
for file in os.listdir(returns_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(returns_path, file))
        all_returns.append(df)
returns = pd.concat(all_returns, ignore_index=True)

# Load sentiment/trend data
reddit = pd.read_csv(reddit_path)
news = pd.read_csv(news_path)
google = pd.read_csv(google_path)

# --------------------------
# 1. Clean and standardize
# --------------------------
for df in [returns, reddit, news, google]:
    df["ticker"] = df["ticker"].astype(str).str.strip().str.upper()
    df["date"] = pd.to_datetime(df["date"]).dt.date

# Rename score columns to avoid collision
reddit = reddit.rename(columns={"sentiment": "reddit_sentiment"})
news = news.rename(columns={"sentiment": "news_sentiment"})
google = google.rename(columns={"trend_score": "google_trend"})

# --------------------------
# 2. Merge all data
# --------------------------
df = returns.copy()
df = df.merge(reddit, on=["date", "ticker"], how="left")
df = df.merge(news, on=["date", "ticker"], how="left")
df = df.merge(google, on=["date", "ticker"], how="left")

# --------------------------
# 3. Optional Forward Fill (limit to 2 days)
# --------------------------
df = df.sort_values(["ticker", "date"])
df["reddit_sentiment"] = df.groupby("ticker")["reddit_sentiment"].ffill(limit=2)
df["news_sentiment"] = df.groupby("ticker")["news_sentiment"].ffill(limit=2)
df["google_trend"] = df.groupby("ticker")["google_trend"].ffill(limit=2)

# --------------------------
# 4. Save final dataset
# --------------------------
output_path = "/home/rupam/DataAlpha/data/raw/processed/final_features.csv"
df.to_csv(output_path, index=False)
print(f"✅ Final dataset saved to {output_path}")

# --------------------------
# 5. Inspect missing data stats
# --------------------------
missing = df[["reddit_sentiment", "news_sentiment", "google_trend"]].isna().mean()
print("🔍 Percentage of missing values:")
print(missing.round(3))


✅ Final dataset saved to /home/rupam/DataAlpha/data/raw/processed/final_features.csv
🔍 Percentage of missing values:
reddit_sentiment    0.855
news_sentiment      0.996
google_trend        1.000
dtype: float64


In [18]:
print(google.head())
print(google.columns)
print(google["ticker"].unique()[:5])


         date  google_trend ticker
0  2019-12-29            29    MMM
1  2020-01-05            25    MMM
2  2020-01-12            25    MMM
3  2020-01-19            25    MMM
4  2020-01-26            63    MMM
Index(['date', 'google_trend', 'ticker'], dtype='object')
['MMM' 'AOS' 'ABT' 'ABBV' 'ACN']
